# World Bank Development Indicators: Clustering, Forecasting & Inequality Analysis

This notebook presents a comprehensive PySpark workflow applied to the World Bank’s World Development Indicators (1960–2022). The objectives are:

1. **Clustering (2020 Development Tiers):** Use K-Means to segment countries into two development groups based on key socio-economic and environmental indicators.  
2. **Forecasting:** Build and evaluate lag-1 linear regression models to predict next-year GDP and life expectancy, using RMSE and R² to assess out-of-sample performance.  
3. **Inequality Modeling:** Apply ElasticNet regression to identify the strongest predictors of income inequality (Gini index) among governance, social spending, and access variables.

All data processing and modelling are performed with Spark’s DataFrame API and MLlib, and the code is structured for scalability—ready to run on HDFS or a standalone CSV. Key parameters (number of clusters, train/test split year, regularization settings) are defined up front to facilitate reproducibility and further experimentation.  


---

In [2]:
pip install pyspark kagglehub --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

import kagglehub

downloading the World Bank - World Development Indicators dataset from Kaggle

## 1. Data Loading & Preprocessing

In [14]:
!kaggle datasets download -d nicolasgonzalezmunoz/world-bank-world-development-indicators
!unzip world-bank-world-development-indicators.zip -d data

Dataset URL: https://www.kaggle.com/datasets/nicolasgonzalezmunoz/world-bank-world-development-indicators
License(s): world-bank
 94%|███████████████████████████████████▊  | 2.00M/2.12M [00:00<00:00, 2.97MB/s]
100%|██████████████████████████████████████| 2.12M/2.12M [00:00<00:00, 2.86MB/s]
Archive:  world-bank-world-development-indicators.zip
  inflating: data/world_bank_development_indicators.csv  


### 1.1 Loading into PySpark

In [6]:
spark = SparkSession.builder.appName("WorldBankIndicators").getOrCreate()
df = spark.read.csv("data/world_bank_development_indicators.csv", header=True, inferSchema=True)
df.show(5) 
df.printSchema()

print("Total rows:", df.count())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 11:31:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/08 11:32:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+------------------+------------+---------+-----------------+------------------+------------------------------+-------------------------+----------------------+------------------------------+--------------------------+------------+-------------------------+------------------+-----------------+------------------+-----------------------+-------------------------------------+-----------------------+------------+--------+--------------------------------+---------------------------+-------------------+--------------+-----------------------------+----------------------------------+---------------------------+--------------------------+---------------------+------------------+----------------------------+-----------------------+--------------------+---------------+---------------------------+----------------------+------------------------------------+------------------------------+-----------------------------------------+----------+----------+----------+---------------

### 1.2 Checking for missing data: 
- drop columns with too many null values

In [8]:
from pyspark.sql.functions import col, isnan, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df.columns
]).show(n=1, truncate=False)

+-------+----+------------------+------------+---------+-----------------+------------------+------------------------------+-------------------------+----------------------+------------------------------+--------------------------+------------+-------------------------+------------------+-----------------+------------------+-----------------------+-------------------------------------+-----------------------+------------+--------+--------------------------------+---------------------------+-------------------+--------------+-----------------------------+----------------------------------+---------------------------+--------------------------+---------------------+--------------+----------------------------+-----------------------+--------------------+---------------+---------------------------+----------------------+------------------------------------+------------------------------+-----------------------------------------+----------+----------+----------+------------------------+----

### 1.3 Filter & Clean Key Columns 
- keep rows with valid GDP, life expectancy, etc. for erarly analysis

In [10]:
df_filtered = df.filter(
    col("GDP_current_US").isNotNull() &
    col("life_expectancy_at_birth").isNotNull() &
    col("access_to_electricity%").isNotNull()
)

Convert Date Column to Year 

In [12]:
from pyspark.sql.functions import year
df_filtered = df_filtered.withColumn("year", year("date"))

In [14]:
df_filtered.cache()
df_filtered.printSchema()
df_filtered.show(5)

root
 |-- country: string (nullable = true)
 |-- date: date (nullable = true)
 |-- agricultural_land%: double (nullable = true)
 |-- forest_land%: double (nullable = true)
 |-- land_area: double (nullable = true)
 |-- avg_precipitation: double (nullable = true)
 |-- trade_in_services%: double (nullable = true)
 |-- control_of_corruption_estimate: double (nullable = true)
 |-- control_of_corruption_std: double (nullable = true)
 |-- access_to_electricity%: double (nullable = true)
 |-- renewvable_energy_consumption%: double (nullable = true)
 |-- electric_power_consumption: double (nullable = true)
 |-- CO2_emisions: double (nullable = true)
 |-- other_greenhouse_emisions: double (nullable = true)
 |-- population_density: double (nullable = true)
 |-- inflation_annual%: double (nullable = true)
 |-- real_interest_rate: double (nullable = true)
 |-- risk_premium_on_lending: double (nullable = true)
 |-- research_and_development_expenditure%: double (nullable = true)
 |-- central_govermen

-----

## 2. Feature Selection & Vector Assembly 

In this section we identify a small set of well‐distributed development indicators to use for clustering.  
We first inspect and drop any rows with missing values in our chosen variables, then assemble them into a single feature vector (`features`) that Spark ML can consume.  
This prepares the data for the subsequent scaling and K-Means clustering steps.  


In [16]:
#selecting some well-distributed development indicators for clustering 
feature_cols = [
    "GDP_current_US",
    "life_expectancy_at_birth",
    "access_to_electricity%",
    "government_expenditure_on_education%",
    "individuals_using_internet%",
    "gini_index",
    "population_density",
    "government_health_expenditure%"
]

In [18]:
# assembling features into a vector 
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_with_features = assembler.transform(df_filtered)
df_with_features = df_with_features.select("country", "year", "features")

In [20]:
df_2020 = df_with_features.filter(col("year") == 2020)

In [22]:
df_filtered.select([
    count(when(col(c).isNull(), c)).alias(c) for c in feature_cols
]).show()

+--------------+------------------------+----------------------+------------------------------------+---------------------------+----------+------------------+------------------------------+
|GDP_current_US|life_expectancy_at_birth|access_to_electricity%|government_expenditure_on_education%|individuals_using_internet%|gini_index|population_density|government_health_expenditure%|
+--------------+------------------------+----------------------+------------------------------------+---------------------------+----------+------------------+------------------------------+
|             0|                       0|                     0|                                2237|                        254|      4991|                53|                          1999|
+--------------+------------------------+----------------------+------------------------------------+---------------------------+----------+------------------+------------------------------+



In [24]:
df_filtered_clean = df_filtered.dropna(subset=feature_cols)

In [26]:
df_with_features = assembler.transform(df_filtered_clean)
df_with_features = df_with_features.select("country", "year", "features")
df_2020 = df_with_features.filter(col("year") == 2020) 

In [28]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(df_2020)
df_scaled = scaler_model.transform(df_2020)

In [30]:
df_scaled.select("country", "scaled_features").show(5, truncate=False)

+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|country  |scaled_features                                                                                                                                                    |
+---------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Albania  |[-0.30979996500054974,0.19909277470310377,0.2534652505266589,-1.1986460979901048,-0.26436091969878883,-0.8033122180179841,-0.19688969999465378,-0.9657165205825639]|
|Argentina|[-0.19482606596741842,0.005502848094597517,0.2534652505266589,0.3874259315685856,0.46111187978821144,1.0974374375849651,-0.5849098801028539,0.5948537149505484]    |
|Armenia  |[-0.3105821308918184,-0.6507970033703324,0.2534652505266589,-1.7199357632183663,-0.031036096715305713,-1.4178

## 3. KMeans Clustering
Here we apply K-Means clustering to the scaled feature vectors in order to segment countries into development tiers. We first fit the model with an arbitrary k, evaluate its silhouette score to gauge cohesion and separation, then sweep k from 2 to 8 to identify the optimal number of clusters.


In [32]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(featuresCol="scaled_features", predictionCol="cluster", k=4, seed=42)
model = kmeans.fit(df_scaled)
df_cluster = model.transform(df_scaled)

25/05/08 11:32:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [34]:
df_cluster.select("country", "cluster").show(20, truncate=False)

+------------------+-------+
|country           |cluster|
+------------------+-------+
|Albania           |3      |
|Argentina         |1      |
|Armenia           |3      |
|Austria           |0      |
|Belarus           |0      |
|Belgium           |0      |
|Bolivia           |1      |
|Brazil            |1      |
|Bulgaria          |3      |
|Burundi           |3      |
|Chile             |1      |
|China             |2      |
|Colombia          |1      |
|Congo, Dem. Rep.  |3      |
|Costa Rica        |1      |
|Croatia           |0      |
|Cyprus            |0      |
|Czechia           |0      |
|Denmark           |0      |
|Dominican Republic|3      |
+------------------+-------+
only showing top 20 rows



In [36]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(
    featuresCol="scaled_features",
    predictionCol="cluster", 
    metricName="silhouette",
    distanceMeasure="squaredEuclidean"
)
silhouette = evaluator.evaluate(df_cluster)
print(f"Silhouette score = {silhouette:.4f}")

Silhouette score = 0.2793


the silhouette score is a measure of how well the data has been clustered, combining cohesion and separation
0.2793 is meaningful as it is above 0, however, under 0.5 indicates moderate separation and frequent overlap 

In [38]:
scores = [] 
for k in range (2, 9):
    km = KMeans(featuresCol="scaled_features", k=k, seed=42)
    mdl = km.fit(df_scaled)
    preds = mdl.transform(df_scaled)
    ev = ClusteringEvaluator(
        featuresCol="scaled_features",
        predictionCol=km.getPredictionCol(), 
        metricName="silhouette"
    )
    score = ev.evaluate(preds)
    scores.append((k, score))

for k, s in scores:
    print(f"k={k}: silhouette={s:.4f}")

k=2: silhouette=0.7363
k=3: silhouette=0.7012
k=4: silhouette=0.2793
k=5: silhouette=0.4350
k=6: silhouette=0.4480
k=7: silhouette=0.2899
k=8: silhouette=0.3108


the silhouette curve indicates the k=2 is the best option 

In [75]:
raw_2020 = df_filtered_clean.filter(col("year") == 2020)

In [77]:
best_k = 2
km = KMeans(featuresCol="scaled_features", k=best_k, seed=42)
model = km.fit(df_scaled)

df_cluster = model.transform(df_scaled)
df_cluster = df_cluster.withColumnRenamed("prediction", "cluster")

df_cluster.printSchema()
df_cluster.select("country", "cluster").show(5, truncate=False)

df_profile = df_cluster.join(raw_2020, on="country", how="inner")
df_profile.groupBy("cluster").avg(
    "GDP_current_US", 
    "life_expectancy_at_birth", 
    "access_to_electricity%", 
    "gini_index"
).show()


root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- cluster: integer (nullable = false)

+---------+-------+
|country  |cluster|
+---------+-------+
|Albania  |0      |
|Argentina|0      |
|Armenia  |0      |
|Austria  |0      |
|Belarus  |0      |
+---------+-------+
only showing top 5 rows

+-------+--------------------+-----------------------------+---------------------------+------------------+
|cluster| avg(GDP_current_US)|avg(life_expectancy_at_birth)|avg(access_to_electricity%)|   avg(gini_index)|
+-------+--------------------+-----------------------------+---------------------------+------------------+
|      1|3.845907969082704E10|           61.647999999999996|         40.725123786926304|              39.3|
|      0|1.080951102375658...|            76.84101177460052|          99.71386902907797|34.725862068965526|
+-------+--------------------+---------------

## 4. Forecasting Module
In this section we prepare time-series data for two forecasting tasks (GDP and life expectancy).  
1. **Lag creation**: We partition by country and order by year, then create one-year lag features for GDP (`GDP_lag1`) and life expectancy (`lifeExp_lag1`), filtering out any rows with missing lags.  
2. **Train/test split**: We split the lagged DataFrame at year 2015 to create out-of-sample test sets.  
3. **Model fitting**: We fit separate linear regression models—first to predict next-year GDP, then next-year life expectancy—evaluating each with RMSE and R².  


In [42]:
from pyspark.sql.window import Window 
from pyspark.sql.functions import lag, col

window_spec = Window.partitionBy("country").orderBy("year")
df_lagged = (
    df_filtered_clean
    .withColumn("GDP_lag1", lag("GDP_current_US", 1).over(window_spec))
    .withColumn("lifeExp_lag1", lag("life_expectancy_at_birth", 1).over(window_spec))
    .filter(col("GDP_lag1").isNotNull() & col("lifeExp_lag1").isNotNull())
)
df_lagged.show(5)

+-------+----------+------------------+----------------+---------+-----------------+------------------+------------------------------+-------------------------+----------------------+------------------------------+--------------------------+------------+-------------------------+------------------+-----------------+------------------+-----------------------+-------------------------------------+-----------------------+----------------+----------------+--------------------------------+---------------------------+-------------------+--------------+-----------------------------+----------------------------------+---------------------------+--------------------------+---------------------+-------------------+----------------------------+-----------------------+--------------------+-----------------+---------------------------+----------------------+------------------------------------+------------------------------+-----------------------------------------+----------+----------+----------+

In [44]:
#Assemble and split for GDP forecasting 
assembler_gdp = VectorAssembler(inputCols=["GDP_lag1"], outputCol="features_gdp")
df_gdp = (
    assembler_gdp.transform(df_lagged)
    .select("country", "year", "features_gdp", "GDP_current_US")
)
train_gdp = df_gdp.filter(col("year") <= 2015)
test_gdp = df_gdp.filter(col("year") > 2015)

In [46]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

lr_gdp = LinearRegression(
    featuresCol="features_gdp",
    labelCol="GDP_current_US",
    predictionCol="prediction_gdp"
)
model_gdp = lr_gdp.fit(train_gdp)
pred_gdp  = model_gdp.transform(test_gdp)

evaluator = RegressionEvaluator(
    labelCol="GDP_current_US",
    predictionCol="prediction_gdp",
    metricName="rmse"
)
rmse_gdp = evaluator.evaluate(pred_gdp)
r2_gdp   = evaluator.setMetricName("r2").evaluate(pred_gdp)

print(f"GDP Forecast → RMSE = {rmse_gdp:.2f}, R² = {r2_gdp:.3f}")

25/05/08 11:33:23 WARN Instrumentation: [181df734] regParam is zero, which might cause numerical instability and overfitting.
25/05/08 11:33:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


GDP Forecast → RMSE = 122169415034.40, R² = 0.998


RMSE = 12,216,941,503.40 
- this is the "typical" error in the predictions. This is reasonably small relative to overall scale
R^2 = 0.998
- this indicates that the model explains 99.8% of the variance, which makes sense because GDP is highly autocorrelated year-to-year

In [48]:
#  Assemble and split for Life Expectancy forecasting
assembler_le = VectorAssembler(inputCols=["lifeExp_lag1"], outputCol="features_le")
df_le = (
    assembler_le.transform(df_lagged)
    .select("country", "year", "features_le", "life_expectancy_at_birth")
)
train_le = df_le.filter(col("year") <= 2015)
test_le  = df_le.filter(col("year") > 2015)

In [50]:
# Train & evaluate a Linear Regression model for Life Expectancy
lr_le = LinearRegression(
    featuresCol="features_le",
    labelCol="life_expectancy_at_birth",
    predictionCol="prediction_le"
)
model_le = lr_le.fit(train_le)
pred_le  = model_le.transform(test_le)

rmse_le = RegressionEvaluator(
    labelCol="life_expectancy_at_birth",
    predictionCol="prediction_le",
    metricName="rmse"
).evaluate(pred_le)
r2_le   = RegressionEvaluator(
    labelCol="life_expectancy_at_birth",
    predictionCol="prediction_le",
    metricName="r2"
).evaluate(pred_le)

print(f"Life Expectancy Forecast → RMSE = {rmse_le:.2f}, R² = {r2_le:.3f}")
pred_le.select("country","year","life_expectancy_at_birth","prediction_le").show(5)


25/05/08 11:33:28 WARN Instrumentation: [dcfe1e09] regParam is zero, which might cause numerical instability and overfitting.


Life Expectancy Forecast → RMSE = 1.33, R² = 0.951
+-------+----+------------------------+-----------------+
|country|year|life_expectancy_at_birth|    prediction_le|
+-------+----+------------------------+-----------------+
|Albania|2016|                   78.86|78.76087033863917|
|Albania|2017|                  79.047|78.95863556308365|
|Albania|2018|                  79.184|79.12984897498698|
|Albania|2019|                  79.282|79.25528339975038|
|Albania|2020|                  76.989|79.34501021454464|
+-------+----+------------------------+-----------------+
only showing top 5 rows



RMSE = 1.33 years 
- one-lag model is off by about 1.3 years in predicting a country's life expectancy, which is quite reasonable
R^2 = 0.951
- the model explains 95.1% of the variance in the test data, reflecting strong year-to-year persistence in life expectancy


## 5. Inequality Modeling

In this section we explore which development indicators are most closely associated with income inequality (Gini index).  
1. **Candidate selection:** We start with a broad set of governance, spending, access, and social indicators.  
2. **Correlation ranking:** We compute Pearson correlations between each candidate and the Gini index, then sort by absolute value to identify the strongest relationships.  
3. **Feature choice:** Based on the ranking, we select the top five predictors for our ElasticNet regression model.  

The code below carries out these steps and lists the top five drivers of inequality, which we will then use in the regression pipeline that follows in Section 5.1.


In [54]:
from pyspark.sql.functions import corr
from pyspark.sql.functions import col

candidates = [
    "government_expenditure_on_education%",
    "government_health_expenditure%",
    "individuals_using_internet%",
    "rule_of_law_estimate",
    "control_of_corruption_estimate",
    "multidimensional_poverty_headcount_ratio%",
    "human_capital_index",
    "access_to_electricity%",
    "GDP_current_US",
    "life_expectancy_at_birth"
]

#correlations
cors = []
for f in candidates:
    r = df_filtered_clean.stat.corr("gini_index", f)
    cors.append((f, r))


cors_sorted = sorted(cors, key=lambda x: abs(x[1]) if x[1] is not None else 0, reverse=True)

# Print them out
for feature, corr_val in cors_sorted:
    print(f"{feature:40s} → corr with Gini = {corr_val:.3f}")

individuals_using_internet%              → corr with Gini = -0.469
rule_of_law_estimate                     → corr with Gini = -0.440
control_of_corruption_estimate           → corr with Gini = -0.388
life_expectancy_at_birth                 → corr with Gini = -0.378
government_health_expenditure%           → corr with Gini = -0.366
access_to_electricity%                   → corr with Gini = -0.307
government_expenditure_on_education%     → corr with Gini = -0.204
human_capital_index                      → corr with Gini = -0.102
multidimensional_poverty_headcount_ratio% → corr with Gini = -0.073
GDP_current_US                           → corr with Gini = 0.042


according to this correlation function, the top five drivers of income inequality are: 
- individuals_using_internet% (corr ≈ –0.469)

- rule_of_law_estimate (corr ≈ –0.440)

- control_of_corruption_estimate (corr ≈ –0.388)

- life_expectancy_at_birth (corr ≈ –0.378)

- government_health_expenditure% (corr ≈ –0.366)

### 5.1 Inequality Regression Pipeline 

In [60]:
ineq_features = [
    "individuals_using_internet%",
    "rule_of_law_estimate",
    "control_of_corruption_estimate",
    "life_expectancy_at_birth",
    "government_health_expenditure%"
]


In [64]:
#dropping any rows missing those predictors/target
raw_drop = df_filtered_clean.na.drop(
    subset=ineq_features + ["gini_index"]
)

In [68]:
#assembling into a feature vector
assembler_ineq = VectorAssembler(
    inputCols=ineq_features, 
    outputCol="features_ineq"
)
df_ineq_all = assembler_ineq.transform(raw_drop)

df_ineq = df_ineq_all.select("country", "year", "features_ineq", "gini_index") 
df_ineq.show(5)

+-------+----+--------------------+----------+
|country|year|       features_ineq|gini_index|
+-------+----+--------------------+----------+
|Albania|2002|[0.390081273,-0.7...|      31.7|
|Albania|2005|[6.043890864,-0.7...|      30.6|
|Albania|2008|[23.86,-0.6000792...|      30.0|
|Albania|2012|[49.4,-0.51983976...|      29.0|
|Albania|2014|[54.3,-0.31097984...|      34.6|
+-------+----+--------------------+----------+
only showing top 5 rows



In [70]:
train_ineq = df_ineq.filter(col("year") <= 2015)
test_ineq  = df_ineq.filter(col("year") > 2015)

# Train
lr_ineq = LinearRegression(
    featuresCol="features_ineq",
    labelCol="gini_index",
    predictionCol="prediction_ineq",
    regParam=0.1,
    elasticNetParam=0.5
)
model_ineq = lr_ineq.fit(train_ineq)

# Predict & Evaluate
preds_ineq = model_ineq.transform(test_ineq)

rmse = RegressionEvaluator(
    labelCol="gini_index", 
    predictionCol="prediction_ineq", 
    metricName="rmse"
).evaluate(preds_ineq)

r2 = RegressionEvaluator(
    labelCol="gini_index", 
    predictionCol="prediction_ineq", 
    metricName="r2"
).evaluate(preds_ineq)

print(f"Inequality Model → RMSE = {rmse:.3f}, R² = {r2:.3f}")


Inequality Model → RMSE = 6.522, R² = 0.154


**Inequality Regression Results**

- **Model:** Linear regression (5 predictors, regParam=0.1, elasticNetParam=0.5)  
- **Out‐of‐sample performance:**  
  - RMSE = 6.52 (Gini points)  
  - R² = 0.154  

> The model explains only ~15% of the year‐to‐year variance in the Gini index, with an average prediction error of about 6.5 Gini points. This relatively low R² is expected given the complexity of income inequality—factors like tax policy, labor market dynamics and demographic shifts are not captured here. Future improvements could include additional predictors, multi‐lag features, nonlinear algorithms (e.g. GBT), or separate models for high‐ vs. low‐development clusters.  


-----

## 6. Summary 



**1. Clustering (2020 Development Tiers)**  
- K‐Means with k=2 gave the highest silhouette (0.7363)  
- **Cluster 0:** High‐development (avg GDP ≈1.08×10¹¹, life exp ≈76.8 yrs, electricity ≈99.7%, Gini ≈34.7)  
- **Cluster 1:** Lower‐development (avg GDP ≈3.85×10¹⁰, life exp ≈61.6 yrs, electricity ≈40.7%, Gini ≈39.3)

**2. Time‐Series Forecasting**  
- **GDP:** Lag‐1 linear model → RMSE ≈12.2 billion, R²=0.998  
- **Life expectancy:** Lag‐1 linear model → RMSE ≈1.33 yrs, R²=0.951  
  - Strong year‐to‐year autocorrelation; future work could add multi‐lag and external predictors.

**3. Inequality Modeling**  
- Predictors: internet access, rule of law, corruption control, life expectancy, health spending  
- ElasticNet regression → RMSE ≈6.52, R²=0.154  
  - Only 15% of variance explained—inequality is driven by many additional factors.

**Next Steps & Scaling**  
- Extend feature sets (tax revenue, labor metrics, demographics) and use non‐linear models  
- Run full pipeline on Lena’s HDFS using PySpark (cache intermediate DataFrames, partition by country)  
- In a production setting, parameter‐tune `k`, `regParam`, and `elasticNetParam` via cross‐validation  

*This notebook demonstrated end-to-end big-data workflows in PySpark: clustering, forecasting, and regression modeling.*  


---

## 7. Appendix and References

### Data Sources  
- **World Bank World Development Indicators (1960–2022)**  
  URL: https://databank.worldbank.org/source/world-development-indicators  
- **Kaggle mirror** (for standalone CSV download)  
  https://www.kaggle.com/datasets/nicolasgonzalezmunoz/world-bank-world-development-indicators  

### Tools & Environment  
- **Apache Spark** 3.3.0 (Spark SQL & MLlib)  
- **PySpark** (Python 3.9)  
- **Hadoop** 3.x client (for HDFS integration on Lena)  
- **Java** 8 or 11 (required by Spark)  
- **Kaggle API** (for local data pulls via `kaggledatasets` package)  

### How to Run on Lena HDFS  
```python
# reference path on UoL Lena
df_raw = spark.read.csv(
    "hdfs:///user/miovi001/worldbank/world_bank_development_indicators.csv",
    header=True,
    inferSchema=True
)
